#### Library Imports

In [7]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib_inline.backend_inline as backend_inline
from torchsummary import summary
backend_inline.set_matplotlib_formats("svg")


#### Pytorch device specific configuration ###
# # Pytorch Gpu Configuration for Cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pytorch Gpu Configuration for directml(AMD GPU)
# import torch_directml

# device = torch_directml.device()

# Set default device
torch.set_default_device(device)

#### Data Import and Data Normalization

In [8]:
# Import Dataset
data = np.loadtxt("../Datasets/mnist_train_small.csv", delimiter=",")

# Extract data and labels
labels = data[:, 0]
data = data[:, 1:]
print("Data shape:", data.shape, "\nLabels shape:", labels.shape)

# Data normalization
dataNorm = data / np.max(data)

# Reshape the data to 2D
dataNorm = dataNorm.reshape(dataNorm.shape[0], 1, 28, 28)

# Print data final shape
print("Data final shape:", dataNorm.shape)

Data shape: (20000, 784) 
Labels shape: (20000,)
Data final shape: (20000, 1, 28, 28)


#### Create Train/Test groups using dataloaders

In [9]:
# Step 1: Convert data into tensors
dataT = torch.tensor(dataNorm).float()
labelsT = torch.tensor(labels).long()

# Step 2: Train and Test split
train_data, test_data, train_labels, test_labels = train_test_split(dataT, labelsT, test_size=0.1)

# Step 3: Convert data into PyTorch Datasets
train_data = TensorDataset(train_data, train_labels)
test_data = TensorDataset(test_data, test_labels)

# Step 4: Create DataLoaders for Train and Test sets
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True, generator=torch.Generator(device=device))
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0], shuffle=False, drop_last=False, generator=torch.Generator(device=device))

In [10]:
# Check the shape of the data in the DataLoader
train_loader.dataset.tensors[0].shape, test_loader.dataset.tensors[0].shape

(torch.Size([18000, 1, 28, 28]), torch.Size([2000, 1, 28, 28]))